# Análisis Multivariante para Decisiones Inmobiliarias: Comprar vs Alquilar

Este notebook presenta un análisis avanzado de los factores que influyen en la decisión de comprar o alquilar una vivienda, utilizando técnicas de análisis multivariante y aprendizaje no supervisado para identificar patrones y relaciones complejas en los datos.

## Objetivos:
- Realizar un análisis exploratorio profundo de factores económicos y personales
- Implementar técnicas de reducción de dimensionalidad (PCA, t-SNE)
- Identificar perfiles de decisión mediante técnicas de clustering
- Visualizar relaciones complejas entre variables
- Generar recomendaciones personalizadas basadas en perfiles

## 1. Importación de Librerías y Configuración

In [ ]:
# Librerías fundamentales
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Preprocesamiento y reducción de dimensionalidad
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA, KernelPCA, SparsePCA, TruncatedSVD
from sklearn.manifold import TSNE, MDS, Isomap, LocallyLinearEmbedding

# Algoritmos de clustering
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN, SpectralClustering
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Configuración
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_theme(style='whitegrid', palette='viridis')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración para reproducibilidad
np.random.seed(42)

print('✅ Librerías importadas correctamente')

## 2. Carga y Exploración Inicial de Datos

In [ ]:
# Cargar el conjunto de datos
try:
    # Intentar cargar desde diferentes fuentes posibles
    try:
        df = pd.read_csv('datos_inmobiliarios.csv')
        print('Datos cargados desde datos_inmobiliarios.csv')
    except FileNotFoundError:
        try:
            df = pd.read_csv('alquilar_comprar.csv')
            print('Datos cargados desde alquilar_comprar.csv')
        except FileNotFoundError:
            # Si no se encuentra el archivo, crear datos sintéticos para demostración
            print('Archivos de datos no encontrados. Creando datos sintéticos para demostración...')
            
            # Generar datos sintéticos para demostración
            np.random.seed(42)
            n_samples = 500
            
            # Factores económicos
            ingresos = np.random.normal(5000, 1500, n_samples)
            precio_vivienda = np.random.normal(250000, 75000, n_samples)
            precio_alquiler = np.random.normal(1200, 400, n_samples)
            tasa_interes = np.random.normal(3.5, 0.8, n_samples)
            inflacion = np.random.normal(2.5, 0.5, n_samples)
            
            # Factores personales
            edad = np.random.normal(35, 10, n_samples)
            tiempo_permanencia = np.random.normal(7, 3, n_samples)
            estabilidad_laboral = np.random.normal(7, 2, n_samples)
            preferencia_propiedad = np.random.normal(6, 2, n_samples)
            aversion_riesgo = np.random.normal(5, 2, n_samples)
            
            # Factores de mercado
            apreciacion_propiedad = np.random.normal(3.0, 1.0, n_samples)
            oferta_vivienda = np.random.normal(5, 2, n_samples)
            demanda_vivienda = np.random.normal(6, 2, n_samples)
            tendencia_mercado = np.random.normal(0, 1, n_samples)
            
            # Factores de ubicación
            calidad_ubicacion = np.random.normal(7, 2, n_samples)
            acceso_servicios = np.random.normal(6, 2, n_samples)
            distancia_trabajo = np.random.normal(15, 8, n_samples)
            
            # Factores fiscales y legales
            beneficios_fiscales = np.random.normal(4, 2, n_samples)
            costos_mantenimiento = np.random.normal(300, 100, n_samples)
            
            # Variables categóricas
            estado_civil = np.random.choice(['Soltero', 'Casado', 'Divorciado'], n_samples, p=[0.4, 0.5, 0.1])
            tiene_hijos = np.random.choice(['Sí', 'No'], n_samples, p=[0.6, 0.4])
            tipo_empleo = np.random.choice(['Tiempo completo', 'Tiempo parcial', 'Autónomo'], n_samples, p=[0.7, 0.1, 0.2])
            zona = np.random.choice(['Urbana', 'Suburbana', 'Rural'], n_samples, p=[0.5, 0.4, 0.1])
            
            # Variable objetivo (simulada)
            # Crear una función que determine la decisión basada en los factores
            def simular_decision(ingresos, precio_vivienda, precio_alquiler, tiempo_permanencia, estabilidad_laboral, preferencia_propiedad):
                ratio_precio_ingreso = precio_vivienda / (ingresos * 12)
                ratio_alquiler_ingreso = precio_alquiler / ingresos
                
                # Factores que favorecen comprar
                puntaje_compra = (
                    (tiempo_permanencia > 5) * 2 + 
                    (estabilidad_laboral > 7) * 1.5 + 
                    (preferencia_propiedad > 7) * 1.5 + 
                    (ratio_alquiler_ingreso > 0.3) * 1 - 
                    (ratio_precio_ingreso > 5) * 2
                )
                
                # Añadir algo de aleatoriedad
                puntaje_compra += np.random.normal(0, 1)
                
                return 'Comprar' if puntaje_compra > 0 else 'Alquilar'
            
            decision = [simular_decision(i, pv, pa, tp, el, pp) 
                       for i, pv, pa, tp, el, pp in zip(ingresos, precio_vivienda, precio_alquiler, 
                                                       tiempo_permanencia, estabilidad_laboral, preferencia_propiedad)]
            
            # Crear el DataFrame
            df = pd.DataFrame({
                'Ingresos_Mensuales': ingresos,
                'Precio_Vivienda': precio_vivienda,
                'Precio_Alquiler': precio_alquiler,
                'Tasa_Interes': tasa_interes,
                'Inflacion': inflacion,
                'Edad': edad,
                'Tiempo_Permanencia_Esperado': tiempo_permanencia,
                'Estabilidad_Laboral': estabilidad_laboral,
                'Preferencia_Propiedad': preferencia_propiedad,
                'Aversion_Riesgo': aversion_riesgo,
                'Apreciacion_Propiedad': apreciacion_propiedad,
                'Oferta_Vivienda': oferta_vivienda,
                'Demanda_Vivienda': demanda_vivienda,
                'Tendencia_Mercado': tendencia_mercado,
                'Calidad_Ubicacion': calidad_ubicacion,
                'Acceso_Servicios': acceso_servicios,
                'Distancia_Trabajo': distancia_trabajo,
                'Beneficios_Fiscales': beneficios_fiscales,
                'Costos_Mantenimiento': costos_mantenimiento,
                'Estado_Civil': estado_civil,
                'Tiene_Hijos': tiene_hijos,
                'Tipo_Empleo': tipo_empleo,
                'Zona': zona,
                'Decision': decision
            })
            
            print('Datos sintéticos creados exitosamente.')
except Exception as e:
    print(f'Error al cargar los datos: {e}')
    raise

In [ ]:
# Exploración inicial de los datos
print('📊 Dimensiones del dataset: {}'.format(df.shape))
print('🔍 Columnas: {}'.format(df.columns.tolist()))

# Información básica
print('📋 Información del dataset:')
df.info()

print('📈 Primeras 5 filas:')
df.head()

In [ ]:
# Estadísticas descriptivas
print('📊 Estadísticas descriptivas:')
df.describe()

In [ ]:
# Verificar valores nulos
print('🔍 Valores nulos por columna:')
nulos = df.isnull().sum()
print(nulos[nulos > 0] if nulos.any() > 0 else 'No hay valores nulos en el dataset.')

# Distribución de la variable objetivo
print('📊 Distribución de la variable objetivo (Decision):')
decision_counts = df['Decision'].value_counts()
print(decision_counts)
print(f'Porcentaje de Comprar: {decision_counts["Comprar"]/len(df)*100:.2f}%')
print(f'Porcentaje de Alquilar: {decision_counts["Alquilar"]/len(df)*100:.2f}%')

# Visualizar la distribución
plt.figure(figsize=(10, 6))
sns.countplot(x='Decision', data=df, palette='viridis')
plt.title('Distribución de Decisiones: Comprar vs Alquilar', fontsize=14)
plt.xlabel('Decisión', fontsize=12)
plt.ylabel('Cantidad', fontsize=12)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)

# Añadir etiquetas con los valores
for i, count in enumerate(decision_counts):
    plt.text(i, count + 5, f'{count} ({count/len(df)*100:.1f}%)', 
             ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

## 3. Análisis Exploratorio de Datos (EDA)

### 3.1 Análisis de Variables Numéricas

In [ ]:
# Identificar variables numéricas y categóricas
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()

print(f'Variables numéricas ({len(numeric_cols)}): {numeric_cols}')
print(f'Variables categóricas ({len(categorical_cols)}): {categorical_cols}')

# Distribución de variables numéricas clave
plt.figure(figsize=(18, 15))

# Seleccionar algunas variables numéricas importantes para visualizar
key_numeric_vars = ['Ingresos_Mensuales', 'Precio_Vivienda', 'Precio_Alquiler', 'Edad', 
                    'Tiempo_Permanencia_Esperado', 'Estabilidad_Laboral', 'Preferencia_Propiedad']

for i, var in enumerate(key_numeric_vars):
    plt.subplot(3, 3, i+1)
    
    # Histograma con KDE
    sns.histplot(df[var], kde=True, color='skyblue')
    
    # Añadir línea vertical para la media
    plt.axvline(df[var].mean(), color='red', linestyle='--', label=f'Media: {df[var].mean():.2f}')
    
    # Añadir línea vertical para la mediana
    plt.axvline(df[var].median(), color='green', linestyle='-.', label=f'Mediana: {df[var].median():.2f}')
    
    plt.title(f'Distribución de {var}')
    plt.legend(fontsize=8)
    plt.tight_layout()

plt.suptitle('Distribución de Variables Numéricas Clave', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para detectar outliers en variables numéricas clave
plt.figure(figsize=(18, 15))

for i, var in enumerate(key_numeric_vars):
    plt.subplot(3, 3, i+1)
    
    # Boxplot por decisión
    sns.boxplot(x='Decision', y=var, data=df, palette='viridis')
    
    plt.title(f'Boxplot de {var} por Decisión')
    plt.xticks(rotation=45)
    plt.tight_layout()

plt.suptitle('Boxplots de Variables Numéricas por Decisión', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 3.2 Análisis de Variables Categóricas

In [ ]:
# Análisis de variables categóricas
plt.figure(figsize=(18, 15))

for i, var in enumerate(categorical_cols):
    if var != 'Decision':  # Excluimos la variable objetivo
        plt.subplot(2, 2, i)
        
        # Crear tabla de contingencia
        cross_tab = pd.crosstab(df[var], df['Decision'])
        cross_tab_pct = pd.crosstab(df[var], df['Decision'], normalize='index') * 100
        
        # Gráfico de barras apiladas
        cross_tab.plot(kind='bar', stacked=True, ax=plt.gca(), colormap='viridis')
        
        plt.title(f'Distribución de Decisiones por {var}')
        plt.xlabel(var)
        plt.ylabel('Cantidad')
        plt.xticks(rotation=45)
        
        # Añadir porcentajes
        for j, category in enumerate(cross_tab.index):
            for k, decision in enumerate(['Alquilar', 'Comprar']):
                if decision in cross_tab.columns:
                    plt.text(j, cross_tab.loc[category, 'Alquilar'] + cross_tab.loc[category, 'Comprar']/2 if k == 1 else cross_tab.loc[category, 'Alquilar']/2, 
                             f'{cross_tab_pct.loc[category, decision]:.1f}%', 
                             ha='center', va='center', color='white' if k == 0 else 'black', fontweight='bold')

plt.tight_layout()
plt.subplots_adjust(top=0.9)
plt.suptitle('Análisis de Variables Categóricas vs Decisión', fontsize=16)
plt.show()

### 3.3 Análisis de Correlación

In [ ]:
# Matriz de correlación para variables numéricas
corr_matrix = df[numeric_cols].corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(14, 12))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Máscara para mostrar solo la mitad inferior

# Crear el mapa de calor
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=.5, cbar_kws={'shrink': .8})

plt.title('Matriz de Correlación de Variables Numéricas', fontsize=16)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Identificar pares de variables con alta correlación (|corr| > 0.7)
high_corr_pairs = []

for i in range(len(corr_matrix.columns)):
    for j in range(i):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((corr_matrix.columns[i], corr_matrix.columns[j], corr_matrix.iloc[i, j]))

print('Pares de variables con alta correlación (|corr| > 0.7):')
for var1, var2, corr in high_corr_pairs:
    print(f'{var1} - {var2}: {corr:.3f}')

# Visualizar las relaciones entre variables altamente correlacionadas
if high_corr_pairs:
    plt.figure(figsize=(15, 5 * len(high_corr_pairs)))
    
    for i, (var1, var2, _) in enumerate(high_corr_pairs):
        plt.subplot(len(high_corr_pairs), 1, i+1)
        
        # Scatter plot con línea de regresión
        sns.regplot(x=var1, y=var2, data=df, scatter_kws={'alpha':0.5}, line_kws={'color':'red'})
        
        plt.title(f'Relación entre {var1} y {var2} (corr = {corr_matrix.loc[var1, var2]:.3f})')
        plt.tight_layout()
    
    plt.show()

## 4. Preprocesamiento de Datos

### 4.1 Codificación de Variables Categóricas

In [ ]:
# Codificación de variables categóricas
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Crear una copia del DataFrame original
df_encoded = df.copy()

# Codificar la variable objetivo con LabelEncoder
le = LabelEncoder()
df_encoded['Decision_encoded'] = le.fit_transform(df_encoded['Decision'])
print(f'Codificación de Decision: {dict(zip(le.classes_, le.transform(le.classes_)))}')

# Aplicar One-Hot Encoding a las variables categóricas (excepto la variable objetivo)
categorical_features = [col for col in categorical_cols if col != 'Decision']

# Usar pandas get_dummies para One-Hot Encoding
df_encoded = pd.get_dummies(df_encoded, columns=categorical_features, drop_first=True)

print('Columnas después de la codificación:')
print(df_encoded.columns.tolist())

# Mostrar las primeras filas del DataFrame codificado
df_encoded.head()

### 4.2 Escalado de Características

In [ ]:
# Comparación de diferentes métodos de escalado
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Seleccionar variables numéricas para escalar
numeric_features = df_encoded.select_dtypes(include=['float64', 'int64']).columns.tolist()
numeric_features.remove('Decision_encoded')  # Excluir la variable objetivo codificada

# Crear DataFrames para cada tipo de escalado
df_standard = df_encoded.copy()
df_minmax = df_encoded.copy()
df_robust = df_encoded.copy()

# Aplicar StandardScaler
std_scaler = StandardScaler()
df_standard[numeric_features] = std_scaler.fit_transform(df_standard[numeric_features])

# Aplicar MinMaxScaler
minmax_scaler = MinMaxScaler()
df_minmax[numeric_features] = minmax_scaler.fit_transform(df_minmax[numeric_features])

# Aplicar RobustScaler
robust_scaler = RobustScaler()
df_robust[numeric_features] = robust_scaler.fit_transform(df_robust[numeric_features])

# Visualizar el efecto de los diferentes escaladores en algunas variables clave
key_vars_to_plot = ['Ingresos_Mensuales', 'Precio_Vivienda', 'Precio_Alquiler']

plt.figure(figsize=(18, 15))

for i, var in enumerate(key_vars_to_plot):
    # Original data
    plt.subplot(4, 3, i+1)
    sns.histplot(df[var], kde=True, color='blue')
    plt.title(f'Original: {var}')
    
    # StandardScaler
    plt.subplot(4, 3, i+4)
    sns.histplot(df_standard[var], kde=True, color='green')
    plt.title(f'StandardScaler: {var}')
    
    # MinMaxScaler
    plt.subplot(4, 3, i+7)
    sns.histplot(df_minmax[var], kde=True, color='red')
    plt.title(f'MinMaxScaler: {var}')
    
    # RobustScaler
    plt.subplot(4, 3, i+10)
    sns.histplot(df_robust[var], kde=True, color='purple')
    plt.title(f'RobustScaler: {var}')

plt.tight_layout()
plt.suptitle('Comparación de Métodos de Escalado', fontsize=16, y=1.02)
plt.show()

In [ ]:
# Seleccionar el método de escalado más apropiado
# Para este análisis, usaremos StandardScaler ya que es adecuado para PCA

# Crear el DataFrame final para el análisis
df_final = df_standard.copy()

print('DataFrame final preparado para análisis multivariante.')
print(f'Dimensiones: {df_final.shape}')
print(f'Columnas: {len(df_final.columns)}')

# Guardar los nombres de las características para su uso posterior
X_features = [col for col in df_final.columns if col != 'Decision' and col != 'Decision_encoded']

# Preparar matrices X e y para el análisis
X = df_final[X_features].values
y = df_final['Decision_encoded'].values

print(f'Matriz X: {X.shape}')
print(f'Vector y: {y.shape}')

## 5. Análisis de Componentes Principales (PCA)

### 5.1 Aplicación de PCA y Análisis de Varianza Explicada

In [ ]:
# Aplicar PCA
pca = PCA()
X_pca = pca.fit_transform(X)

# Varianza explicada por cada componente
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Visualizar la varianza explicada
plt.figure(figsize=(12, 6))

# Gráfico de barras para la varianza explicada por componente
plt.subplot(1, 2, 1)
plt.bar(range(1, len(explained_variance) + 1), explained_variance, alpha=0.7, color='skyblue')
plt.plot(range(1, len(explained_variance) + 1), explained_variance, 'o-', color='red')
plt.title('Varianza Explicada por Componente')
plt.xlabel('Componente Principal')
plt.ylabel('Proporción de Varianza Explicada')
plt.xticks(range(1, len(explained_variance) + 1))
plt.grid(True, alpha=0.3)

# Gráfico de línea para la varianza acumulada
plt.subplot(1, 2, 2)
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 'o-', color='green')
plt.axhline(y=0.8, color='red', linestyle='--', label='80% Varianza')
plt.axhline(y=0.9, color='orange', linestyle='--', label='90% Varianza')
plt.title('Varianza Acumulada')
plt.xlabel('Número de Componentes')
plt.ylabel('Varianza Acumulada')
plt.xticks(range(1, len(cumulative_variance) + 1))
plt.grid(True, alpha=0.3)
plt.legend()

plt.tight_layout()
plt.show()

# Determinar el número óptimo de componentes (80% y 90% de varianza)
n_components_80 = np.where(cumulative_variance >= 0.8)[0][0] + 1
n_components_90 = np.where(cumulative_variance >= 0.9)[0][0] + 1

print(f'Número de componentes para explicar el 80% de la varianza: {n_components_80}')
print(f'Número de componentes para explicar el 90% de la varianza: {n_components_90}')

# Mostrar la varianza explicada por cada componente
print('Varianza explicada por cada componente:')
for i, var in enumerate(explained_variance[:10]):  # Mostrar solo los primeros 10 componentes
    print(f'PC{i+1}: {var:.4f} ({cumulative_variance[i]:.4f} acumulado)')

### 5.2 Análisis de Cargas (Loadings) de los Componentes

In [ ]:
# Analizar las cargas (loadings) de los componentes principales
loadings = pca.components_

# Crear un DataFrame con las cargas
loadings_df = pd.DataFrame(loadings.T, index=X_features, columns=[f'PC{i+1}' for i in range(loadings.shape[0])])

# Mostrar las cargas de los primeros 3 componentes
print('Cargas de los primeros 3 componentes principales:')
loadings_df.iloc[:, :3].sort_values(by='PC1', ascending=False)

In [ ]:
# Visualizar las cargas de los primeros 3 componentes
plt.figure(figsize=(18, 15))

for i in range(3):  # Para los primeros 3 componentes
    plt.subplot(3, 1, i+1)
    
    # Ordenar las cargas por valor absoluto
    component = loadings_df.iloc[:, i].sort_values(ascending=False)
    
    # Crear un gráfico de barras horizontales
    bars = plt.barh(component.index, component.values, color=['red' if x < 0 else 'green' for x in component.values])
    
    # Añadir etiquetas con los valores
    for bar in bars:
        width = bar.get_width()
        label_x_pos = width + 0.01 if width > 0 else width - 0.01
        plt.text(label_x_pos, bar.get_y() + bar.get_height()/2, f'{width:.3f}', 
                 va='center', ha='left' if width > 0 else 'right', fontsize=8)
    
    plt.title(f'Cargas del Componente Principal {i+1}')
    plt.axvline(x=0, color='black', linestyle='-', alpha=0.3)
    plt.grid(True, alpha=0.3)
    plt.tight_layout()

plt.suptitle('Análisis de Cargas de los Componentes Principales', fontsize=16, y=1.02)
plt.tight_layout()
plt.show()

### 5.3 Visualización de Datos en el Espacio de Componentes Principales

In [ ]:
# Crear un DataFrame con los componentes principales
pca_df = pd.DataFrame(X_pca[:, :3], columns=['PC1', 'PC2', 'PC3'])
pca_df['Decision'] = df['Decision']

# Visualización 2D (PC1 vs PC2)
plt.figure(figsize=(12, 10))

# Scatter plot con colores por decisión
sns.scatterplot(x='PC1', y='PC2', hue='Decision', data=pca_df, palette='viridis', s=100, alpha=0.7)

# Añadir flechas para las variables originales (biplot)
# Seleccionar solo algunas variables importantes para mayor claridad
key_features = ['Ingresos_Mensuales', 'Precio_Vivienda', 'Precio_Alquiler', 'Edad', 
                'Tiempo_Permanencia_Esperado', 'Estabilidad_Laboral', 'Preferencia_Propiedad']

# Encontrar los índices de estas características en X_features
key_indices = [X_features.index(feat) for feat in key_features if feat in X_features]

# Factor de escala para las flechas
scale_factor = 5

for i in key_indices:
    plt.arrow(0, 0, loadings[0, i] * scale_factor, loadings[1, i] * scale_factor, 
              head_width=0.2, head_length=0.2, fc='blue', ec='blue', alpha=0.6)
    plt.text(loadings[0, i] * scale_factor * 1.2, loadings[1, i] * scale_factor * 1.2, 
             X_features[i], color='blue', ha='center', va='center', fontsize=10)

plt.title('Visualización PCA: PC1 vs PC2', fontsize=16)
plt.xlabel(f'PC1 ({explained_variance[0]:.2%} varianza)', fontsize=12)
plt.ylabel(f'PC2 ({explained_variance[1]:.2%} varianza)', fontsize=12)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.3)
plt.axvline(x=0, color='gray', linestyle='--', alpha=0.3)
plt.grid(True, alpha=0.3)
plt.legend(title='Decisión')
plt.tight_layout()
plt.show()